In [ ]:
# 수동 중단 설정
import asyncio
from contextlib import aclosing
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph
from langchain_core.messages import HumanMessage

async def interrupt(event):
    await asyncio.sleep(2)
    event.set()
    print("중단 신호")
    
async def main():
    builder=StateGraph(State)
    builder.add_node('select_tools' ,select_tools)
    builder.add_node('model' ,model_node)
    builder.add_node('tools' ,ToolNode(tools))
    builder.add_edge(START ,'select_tools')
    builder.add_edge('select_tools' ,'model')
    builder.add_conditional_edges('model' ,tools_condition)
    builder.add_edge('tools' ,'model')
    graph =builder.compile(checkpointer=MemorySaver())
    
    event=asyncio.Event()
    input={
        'messages':[
            HumanMessage(
                '미국 제 30대 대통령의 사망 당시 나이는 몇 살이었나요?'
            )
        ]
    }
    
    config={'configurable':{'thread_id' :'1'}}
    interrupt_task =asyncio.create_task(interrupt(event))
    
    async with aclosing(graph.astream(input ,config)) as stream:
        async for chunk in stream:
            if event.is_set():
                print("\n --- 중단됨 ---")
                break
            else:
                print(chunk)
                await interrupt_task
                
                
if __name__=='__main__':
    asyncio.run(main())

In [ ]:
# authorize 패턴
input={
    'messages':[
        HumanMessage(
            '미국 제30대 대통령의 사망 당시 나이?'
        )
    ]
}

config={"configurable":{"thread_id" :"1"}}

output=graph.stream(input ,config ,interrupt_before=["model"])
for c in output:
    print(c)

In [ ]:
# 재개
input={
    'messages':[
        HumanMessage(
            '미국 제30대 대통령의 사망 당시 나이?'
        )
    ]
}

config={"configurable":{"thread_id" :"1"}}

output=graph.stream(input ,config ,interrupt_before=["model"])
for c in output:
    print(c)
    
output= graph.stream(None ,config)
